<a href="https://colab.research.google.com/github/masubi/grover/blob/master/Grover_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#!pip install bert-pytorch
!pip install absl-py

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


/content/gdrive/My Drive/NLP_Projects/grover-master


In [48]:
!pwd

/content/gdrive/My Drive/NLP_Projects/grover-master


In [0]:
# !python3 download_model.py base #run if it hasn't downloaded model yet

In [0]:
"""
For discrimination finetuning (e.g. saying whether or not the generation is human/grover)
"""
import json
import os

import numpy as np
import tensorflow as tf
from tensorflow.python.lib.io import file_io

from lm.dataloader import classification_convert_examples_to_features, classification_input_fn_builder
from lm.modeling import classification_model_fn_builder, GroverConfig
from lm.utils import _save_np
from sample.encoder import get_encoder

In [55]:
flags = tf.flags
FLAGS = flags.FLAGS

#
# Clearing flags in this section so can rerun,  FLAGS are global which is why
# error kept occurring 
#
configListToRemove = ["config_file", 
                      "input_data", 
                      "additional_data", 
                      "output_dir",
                      "init_checkpoint",
                      "max_seq_length",
                      "iterations_per_loop",
                      "batch_size",
                      "max_training_examples",
                      "do_train",
                      "predict_val",
                      "predict_test",
                      "num_train_epochs",
                      "warmup_proportion",
                      "adafactor",
                      "learning_rate",
                      "use_tpu",
                      "tpu_name",
                      "tpu_zone",
                      "gcp_project",
                      "master",
                      "num_tpu_cores",
                      "f"
                      ]
                
for name in list(flags.FLAGS):
  if(name in configListToRemove):
    delattr(flags.FLAGS,name)
    print("pre-delAttr: " + name)

#for name in list(flags.FLAGS):
#  
## Required parameters

flags.DEFINE_string(
    "config_file", 'lm/configs/base.json',
    "The config json file corresponding to the pre-trained news model. "
    "This specifies the model architecture.")
      
flags.DEFINE_string(
    "input_data", '/content/gdrive/My Drive/NLP_Projects/grover-master/discrimination/input/',
    "The input data dir. Should contain the .tsv files (or other data files) for the task.")

flags.DEFINE_string(
    "additional_data", None,
    "Should we provide additional input data? maybe.")

flags.DEFINE_string(
    "output_dir", '/content/gdrive/My Drive/NLP_Projects/grover-master/discrimination/output/',
    "The output directory where the model checkpoints will be written.")

## Other parameters
flags.DEFINE_string(
    "init_checkpoint", None,
    "Initial checkpoint (usually from a pre-trained model).")

flags.DEFINE_integer(
    "max_seq_length", 1024,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded. Must match data generation.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_integer("batch_size", 32, "Batch size used")

flags.DEFINE_integer("max_training_examples", -1, "if you wanna limit the number")

flags.DEFINE_bool("do_train", True, "Whether to run training.")

flags.DEFINE_bool("predict_val", False, "Whether to run eval on the dev set.")

flags.DEFINE_bool(
    "predict_test", False,
    "Whether to run the model in inference mode on the test set.")

flags.DEFINE_float("num_train_epochs", 3.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

flags.DEFINE_bool("adafactor", False, "Whether to run adafactor")

flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

flags.DEFINE_string(
    "tpu_name", None,
    "The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")

flags.DEFINE_string(
    "tpu_zone", None,
    "[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_integer(
    "num_tpu_cores", 8,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

tf.app.flags.DEFINE_string('f', '', 'kernel')
print("flags loaded")

pre-delAttr: config_file
pre-delAttr: input_data
pre-delAttr: additional_data
pre-delAttr: output_dir
pre-delAttr: init_checkpoint
pre-delAttr: max_seq_length
pre-delAttr: iterations_per_loop
pre-delAttr: batch_size
pre-delAttr: max_training_examples
pre-delAttr: do_train
pre-delAttr: predict_val
pre-delAttr: predict_test
pre-delAttr: num_train_epochs
pre-delAttr: warmup_proportion
pre-delAttr: adafactor
pre-delAttr: learning_rate
pre-delAttr: use_tpu
pre-delAttr: tpu_name
pre-delAttr: tpu_zone
pre-delAttr: gcp_project
pre-delAttr: master
pre-delAttr: num_tpu_cores
pre-delAttr: f
flags loaded


In [0]:
def _flatten_and_tokenize_metadata(encoder, item):
    """
    Turn the article into tokens
    :param item: Contains things that need to be tokenized

    fields are ['domain', 'date', 'authors', 'title', 'article', 'summary']
    :return: dict
    """
    metadata = []
    for key in ['domain', 'date', 'authors', 'title', 'article']:
        val = item.get(key, None)
        if val is not None:
            metadata.append(encoder.__dict__[f'begin_{key}'])
            metadata.extend(encoder.encode(val))
            metadata.append(encoder.__dict__[f'end_{key}'])
    return metadata

In [0]:

LABEL_LIST = ['machine', 'human']
LABEL_INV_MAP = {label: i for i, label in enumerate(LABEL_LIST)}

tf.logging.set_verbosity(tf.logging.INFO)




In [56]:

def checkFileConfigs():
  if tf.gfile.Exists(FLAGS.output_dir):
      print(f"The output directory {FLAGS.output_dir} exists!")
      #if FLAGS.do_train:
      #    print("EXITING BECAUSE DO_TRAIN is true", flush=True)
      #       return
      for split in ['val', 'test']:
          if tf.gfile.Exists(os.path.join(FLAGS.output_dir, f'{split}-probs.npy')) and getattr(FLAGS,
                                                                                                f'predict_{split}'):
              print(f"EXITING BECAUSE {split}-probs.npy exists", flush=True)
              return
  elif not FLAGS.do_train:
      print("EXITING BECAUSE DO_TRAIN IS FALSE AND PATH DOESNT EXIST")
      return
  #else:
      #tf.gfile.MakeDirs(FLAGS.output_dir)
  print("complete checking")

checkFileConfigs()

complete checking


In [57]:
news_config = GroverConfig.from_json_file(FLAGS.config_file)

# TODO might have to change this
encoder = get_encoder()
examples = {'train': [], 'val': [], 'test': []}
np.random.seed(123456)
tf.logging.info("*** Parsing files ***")
with tf.gfile.Open(FLAGS.input_data, "r") as f:
    for l in f:
        item = json.loads(l)

        # This little hack is because we don't want to tokenize the article twice
        context_ids = _flatten_and_tokenize_metadata(encoder=encoder, item=item)
        examples[item['split']].append({
            'info': item,
            'ids': context_ids,
            'label': item['label'],
        })
        assert item['label'] in LABEL_INV_MAP

additional_data = {'machine': [], 'human': []}
if FLAGS.additional_data is not None:
    print("NOW WERE LOOKING AT ADDITIONAL INPUT DATA", flush=True)
    with tf.gfile.Open(FLAGS.additional_data, "r") as f:
        for l in f:
            item = json.loads(l)
            # This little hack is because we don't want to tokenize the article twice
            context_ids = _flatten_and_tokenize_metadata(encoder=encoder, item=item)
            additional_data[item['label']].append({
                'info': item,
                'ids': context_ids,
                'label': item['label'],
            })

tf.logging.info("*** Done parsing files ***")

INFO:tensorflow:*** Parsing files ***
INFO:tensorflow:*** Done parsing files ***


In [0]:
 
if FLAGS.max_training_examples > 0:
  print("*** Setting max_training_examples ***", flush=True)
  examples_by_label = {'human': [], 'machine': []}
  for x in examples['train']:
      examples_by_label[x['label']].append(x)

  new_examples = []
  print("Unique machine examples: {} -> {}".format(len(examples_by_label['machine']),
                                                    FLAGS.max_training_examples), flush=True)
  machine_ex_to_keep = examples_by_label['machine'][:FLAGS.max_training_examples]

  # So we just cut down on the TRUE machine examples. now lets try adding in additional examples
  # examples_by_label['human'].extend(additional_data['human'])

  if len(additional_data['machine']) > 0:
      amount_to_add = len(examples_by_label['human']) - len(machine_ex_to_keep)
      if amount_to_add > 0:
          machine_ex_to_keep.extend(additional_data['machine'][:amount_to_add])

  for i, human_ex in enumerate(examples_by_label['human']):
      new_examples.append(human_ex)
      new_examples.append(machine_ex_to_keep[i % len(machine_ex_to_keep)])

  print("Length of examples: {} -> {}".format(len(examples['train']), len(new_examples)), flush=True)
  examples['train'] = new_examples
  print("*** Done Setting max_training_examples")



In [64]:

# Training
if FLAGS.do_train:
    print("examples: " + str(len(examples['train'])))
    #num_train_steps = int((len(examples['train']) / FLAGS.batch_size) * FLAGS.num_train_epochs)
    num_train_steps = int((len(examples['train']) / 2) * FLAGS.num_train_epochs)
    num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)
    assert num_train_steps > 0
else:
    num_train_steps = None
    num_warmup_steps = None


examples: 0


In [0]:

# Boilerplate
tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    #model_dir="./discrimination/output/checkpoint/",
    save_checkpoints_steps=FLAGS.iterations_per_loop,
    keep_checkpoint_max=None,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

model_fn = classification_model_fn_builder(
    news_config,
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    num_labels=len(LABEL_LIST),
    pool_token_id=encoder.begin_summary,
    adafactor=FLAGS.adafactor
)

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.batch_size,
    eval_batch_size=FLAGS.batch_size,
    predict_batch_size=FLAGS.batch_size,
    params={'model_dir': FLAGS.output_dir}
)